# Building Apps with Streamlit

Streamlit is an open-source Python library that makes it easy to create and share beautiful, custom web apps for machine learning and data science.

We will use streamlit to build our LLM applications, but first, let's take a look at how to use it.

![](images/streamlit.gif)


## How to run a Streamlit App

You'll need to save the code for your app in a `.py` Python file.

Then to run a Streamlit app, use the following command in your terminal:

```
streamlit run path/to/your_script.py
```

Below are some examples demonstrating Streamlit features.

## Simple streamlit examples

### Example 1: Simple Line Chart

This code below creates an App that shows a simple line chart of a sine wave.

Copy and paste this code to a new file called `simple_line_chart.py`. Remember to save the file!

In [ ]:
# %%writefile simple_line_chart.py

import numpy as np
import streamlit as st

st.title("Our First Streamlit App")
st.write("The description of our app, or some explanatory text.")

x = np.arange(0, 2 * np.pi, 0.01)
y = np.sin(x)
st.line_chart(y)


Run the following command in your **terminal** to start the App:

```python
uv run streamlit run path/to/simple_line_chart.py
```

Alternatively, we have written a helper function to save you typing the path to your file.

```python
uv run invoke simple-line-chat

## Example 2: Interactive Line Chart
In this example, we add interactivity using sliders to adjust the amplitude and period of the sine wave.

In [ ]:
# %%writefile interaction.py

import numpy as np
import streamlit as st

st.title("Interactive Line Chart")
st.write("Adjust the sliders to modify the sine wave.")

amplitude = st.slider("Amplitude", min_value=0, max_value=10, value=1)
period = st.slider("Period", min_value=0, max_value=10, value=1)

st.write(f"This is a sine wave with amplitude **{amplitude}** and period **{period}**.")

x = np.arange(0, 2 * np.pi, 0.01) * period
y = amplitude * np.sin(x)
st.line_chart(y)


After creating `interaction.py`, run the following command in your terminal to open the App.

```python
uv run invoke interaction
```

## Example 3: Using a Sidebar
This example moves the sliders into a sidebar to keep the main interface cleaner.

In [ ]:
# %%writefile sidebar.py
 
import numpy as np
import streamlit as st

st.title("Sidebar Controls")
st.write("Use the sidebar sliders to modify the sine wave.")

st.sidebar.markdown("# Controls")
amplitude = st.sidebar.slider("Amplitude", min_value=0, max_value=10, value=1)
period = st.sidebar.slider("Period", min_value=0, max_value=10, value=1)

st.write(f"This is a sine wave with amplitude **{amplitude}** and period **{period}**.")

x = np.arange(0, 2 * np.pi, 0.01)
y = amplitude * np.sin(x * period)
st.line_chart(y)


After creating `sidebar.py`, run the following command in your terminal to open the App.

```python
uv run invoke sidebar

We're now ready to start making apps using the **LangChain** API.

## Introduction to LangChain

In the previous notebook, we explored how to interact directly with language models, like OpenAI's GPT or Google's Gemini (Vertex AI). While these APIs are powerful, working with them directly can become repetitive and cloud-specific.

LangChain is a Python-based framework that simplifies the use of language models, providing a cloud-agnostic interface for interacting with providers like Google Vertex AI and (Azure) OpenAI. With LangChain, you can focus on what you want the model to do, not how to interact with different APIs.

**Why Use LangChain?**
- Cloud-Agnostic: Switch between providers (e.g., OpenAI, Azure, Google) with minimal changes to your code.
- Simplified Usage: LangChain abstracts away API-specific details, letting you use models in a more consistent and straightforward way.
- Scalability: As your projects grow, LangChain makes it easier to manage interactions with models.


Let's import the necessary libraries and create a client for our LLM.

In [ ]:
from llm_in_production.llm import instantiate_langchain_model
import dotenv
import pandas as pd


dotenv.load_dotenv()

# Here we create the client. 
# Make sure you select the LLM provider that corresponds to the one you are using in this course!
client = instantiate_langchain_model(
    # llm_provider="azure",
    llm_provider="gcp",
)
client.model_name

To use the LangChain client, you can call the `invoke` method. This takes a prompt as input and generates text.

The prompt can be a simple **instruction** to follow:

In [ ]:
response = client.invoke("Hello, tell me about building Apps with streamlit.")

print(response.content)

Or, you can structure your prompt with `role`s.

- `system`: This contains the system instructions the model should follow during the conversation.
- `user`:  This means that the message is something the user said.
- `assistant`: This means that the message is something the assistant/model said.

In [ ]:
system_prompt = "You explain Python concepts in a simple and clear manner."
user_message = "Can you explain how to build streamlit apps?"

messages = [{"role": "system", "content": system_prompt},
           {"role": "user", "content": user_message}]


response = client.invoke(
    input=messages,
    temperature=1.0)

print(response.content)

## Exercise 4: A Streamlit Chat App

Streamlit has great [in-built functionality for building chat apps](https://docs.streamlit.io/develop/tutorials/chat-and-llm-apps/build-conversational-apps).

The example below can be used to create a very simple chatbot.

In [ ]:
# %%writefile simple_chatbot.py

import streamlit as st
from llm_in_production.llm import instantiate_langchain_model
import dotenv

# Step 1: Set up Streamlit UI
title = "Simple Chatbot"
st.set_page_config(
    page_title=title,
    page_icon="👋",
)
st.title(title)

# Step 2: Create LLM Client
dotenv.load_dotenv()

# Here we create the client. 
# Make sure you select the LLM provider that corresponds to the one you are using in this course!
client = instantiate_langchain_model(
    # llm_provider="azure",
    llm_provider="gcp",
)
st.write("Powered by", client.model_name)


# Step 3: Initialize Chat History
if "messages" not in st.session_state:
    st.session_state.messages = []

# Step 4: Display Chat History
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Step 5: Accept User Input
prompt = st.chat_input("Say something:")

if prompt:
    # Add user message to chat history
    input_message = {"role": "user", "content": prompt}
    st.session_state.messages.append(input_message)

    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)

    # Simple response logic
    response_message = client.invoke(st.session_state.messages).content
    
    # Display chatbot response
    with st.chat_message("assistant"):
        st.markdown(response_message)
    
    # Store assistant response
    st.session_state.messages.append(
        {"role": "assistant", "content": response_message}
    )


After creating `simple_chatbot.py`, run the following command in your terminal to open the App.

```python
uv run invoke simple-chatbot

## Streamlit LLM Apps

Now, that you know Streamlit and LangChain, you've all the tools you need to build your own LLM Chatbot App!

Run the following command in your terminal to get started!

```python
uv run invoke chatot
```
---